<a href="https://colab.research.google.com/github/Mahipalsingh6695/stats/blob/main/xGboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.0 MB/s eta 0:00:00


In [2]:
import optuna

In [3]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.3
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
alembic                          1.13.1
altair                           4.2.2
annotated-types                  0.6.0
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
be

In [4]:
import optuna
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [5]:
data=pd.read_csv("https://raw.githubusercontent.com/Chandrakant817/Admission-Prediction/main/Admission_Prediction.csv")

In [6]:
data.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,NaN,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         500 non-null    int64  
 1   GRE Score          485 non-null    float64
 2   TOEFL Score        490 non-null    float64
 3   University Rating  485 non-null    float64
 4   SOP                500 non-null    float64
 5   LOR                500 non-null    float64
 6   CGPA               500 non-null    float64
 7   Research           500 non-null    int64  
 8   Chance of Admit    500 non-null    float64
dtypes: float64(7), int64(2)
memory usage: 35.3 KB


In [8]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Serial No.,500.0,250.500000,144.481833,1.00,125.7500,250.50,375.25,500.00
GRE Score,485.0,316.558763,11.274704,290.00,308.0000,317.00,325.00,340.00
TOEFL Score,490.0,107.187755,6.112899,92.00,103.0000,107.00,112.00,120.00
University Rating,485.0,3.121649,1.146160,1.00,2.0000,3.00,4.00,5.00
SOP,500.0,3.374000,0.991004,1.00,2.5000,3.50,4.00,5.00
LOR,500.0,3.484000,0.925450,1.00,3.0000,3.50,4.00,5.00
CGPA,500.0,8.576440,0.604813,6.80,8.1275,8.56,9.04,9.92
Research,500.0,0.560000,0.496884,0.00,0.0000,1.00,1.00,1.00
Chance of Admit,500.0,0.721740,0.141140,0.34,0.6300,0.72,0.82,0.97


In [9]:
data.isnull().sum()

Serial No.            0
GRE Score            15
TOEFL Score          10
University Rating    15
SOP                   0
LOR                   0
CGPA                  0
Research              0
Chance of Admit       0
dtype: int64

In [10]:
data.isnull().sum().sum()

40

In [11]:
data["GRE Score"]=data["GRE Score"].fillna(data["GRE Score"].median())
data["TOEFL Score"]=data["TOEFL Score"].fillna(data["TOEFL Score"].median())
data["University Rating"]=data["University Rating"].fillna(data["University Rating"].median())

In [12]:
data.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [13]:
data.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,317.0,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65


In [14]:
X=data.drop(["Serial No.","Chance of Admit"],axis=1)

In [15]:
y=data["Chance of Admit"]

In [16]:
X.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337.0,118.0,4.0,4.5,4.5,9.65,1
1,324.0,107.0,4.0,4.0,4.5,8.87,1
2,317.0,104.0,3.0,3.0,3.5,8.00,1
3,322.0,110.0,3.0,3.5,2.5,8.67,1
4,314.0,103.0,2.0,2.0,3.0,8.21,0


In [17]:
y.head()

0    0.92
1    0.76
2    0.72
3    0.80
4    0.65
Name: Chance of Admit, dtype: float64

In [18]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=25)

In [19]:
sc=StandardScaler()

In [20]:
X_train_sc=sc.fit_transform(X_train)

In [21]:
X_test_sc=sc.transform(X_test)

In [22]:
X.isnull().sum()

GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
dtype: int64

In [23]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   GRE Score          500 non-null    float64
 1   TOEFL Score        500 non-null    float64
 2   University Rating  500 non-null    float64
 3   SOP                500 non-null    float64
 4   LOR                500 non-null    float64
 5   CGPA               500 non-null    float64
 6   Research           500 non-null    int64  
dtypes: float64(6), int64(1)
memory usage: 27.5 KB


In [24]:
X["GRE Score"]

0      337.0
1      324.0
2      317.0
3      322.0
4      314.0
       ...  
495    332.0
496    337.0
497    330.0
498    312.0
499    327.0
Name: GRE Score, Length: 500, dtype: float64

In [25]:
for col in X.columns:
    print(f"{col} {X[col].apply(lambda x: isinstance(x,str)).sum()}")

GRE Score 0
TOEFL Score 0
University Rating 0
SOP 0
LOR 0
CGPA 0
Research 0


In [26]:
X.isnull().sum()

GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
dtype: int64

In [27]:
y.isnull().sum()

0

In [28]:
def objective(trail,data=X,target=y):
    train_x,test_x,train_y,test_y=train_test_split(data,target,test_size=0.25,random_state=25)
    param={
      'tree_method': 'gpu_hist',
      'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
      'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),
      'colsample_bytree' :trail.suggest_categorical('colsample_bytree',[.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
      'subsample' :trail.suggest_categorical('subsample',[.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
      'learning_rate' : trail.suggest_categorical('learning_rate' , [.00001,.0003,.008,.02,.01,1,8]),
      'n_estimators' :300,
      'max_depth' :trail.suggest_categorical('max_depth',[3,4,5,6,7,8,9,10,11,12]),
      'random_state' :trail.suggest_categorical('random_state',[10,20,30,2000,3454,243123]),
      'min_child_weight' :trail.suggest_int('min_child_weight',1,200)
      }

    model=xgb.XGBRegressor(**param)
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],verbose=True)
    pred=model.predict(test_x)
    mse=mean_squared_error(test_y,pred)
    return mse


In [67]:
find_params=optuna.create_study()
find_params.optimize(objective,n_trials=15)
find_params.best_trial.params

[I 2024-02-12 18:10:56,952] A new study created in memory with name: no-name-00b8e5be-4b57-492b-90a1-940a729b1703


[0]	validation_0-rmse:0.18275
[1]	validation_0-rmse:0.70399
[2]	validation_0-rmse:4.69018
[3]	validation_0-rmse:31.29513
[4]	validation_0-rmse:209.49959
[5]	validation_0-rmse:1401.51695
[6]	validation_0-rmse:9326.83490
[7]	validation_0-rmse:62443.00013
[8]	validation_0-rmse:415088.19290
[9]	validation_0-rmse:2772485.53735
[10]	validation_0-rmse:18415982.23123
[11]	validation_0-rmse:122928190.09406
[12]	validation_0-rmse:823002872.78163
[13]	validation_0-rmse:5525843902.18719
[14]	validation_0-rmse:36824899922.44373
[15]	validation_0-rmse:247370873767.55237
[16]	validation_0-rmse:1649615794251.08325
[17]	validation_0-rmse:11028259627025.81445
[18]	validation_0-rmse:73705707392675.17188
[19]	validation_0-rmse:493320309472159.50000
[20]	validation_0-rmse:3293993155845541.00000
[21]	validation_0-rmse:21987677142018896.00000
[22]	validation_0-rmse:146528560750724896.00000
[23]	validation_0-rmse:981279216771888384.00000
[24]	validation_0-rmse:6548024781286121472.00000
[25]	validation_0-rmse:

<ipython-input-28-bc99d19b1cf4>:5: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

<ipython-input-28-bc99d19b1cf4>:6: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[18:10:56] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"




[55]	validation_0-rmse:inf
[56]	validation_0-rmse:inf
[57]	validation_0-rmse:inf
[58]	validation_0-rmse:inf
[59]	validation_0-rmse:inf
[60]	validation_0-rmse:inf
[61]	validation_0-rmse:inf
[62]	validation_0-rmse:inf
[63]	validation_0-rmse:inf
[64]	validation_0-rmse:inf
[65]	validation_0-rmse:inf
[66]	validation_0-rmse:inf
[67]	validation_0-rmse:inf
[68]	validation_0-rmse:inf
[69]	validation_0-rmse:inf
[70]	validation_0-rmse:inf
[71]	validation_0-rmse:inf
[72]	validation_0-rmse:inf
[73]	validation_0-rmse:inf
[74]	validation_0-rmse:inf
[75]	validation_0-rmse:inf
[76]	validation_0-rmse:inf
[77]	validation_0-rmse:inf
[78]	validation_0-rmse:inf
[79]	validation_0-rmse:inf
[80]	validation_0-rmse:inf
[81]	validation_0-rmse:inf
[82]	validation_0-rmse:inf
[83]	validation_0-rmse:inf
[84]	validation_0-rmse:inf
[85]	validation_0-rmse:inf
[86]	validation_0-rmse:inf
[87]	validation_0-rmse:inf
[88]	validation_0-rmse:inf
[89]	validation_0-rmse:inf
[90]	validation_0-rmse:inf
[91]	validation_0-rmse:inf
[

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[18:10:57] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"


[I 2024-02-12 18:10:57,997] Trial 0 finished with value: 7.793700683273691e+73 and parameters: {'lambda': 6.223112732635033, 'alpha': 0.004510664607902029, 'colsample_bytree': 0.7, 'subsample': 0.4, 'learning_rate': 8, 'max_depth': 10, 'random_state': 30, 'min_child_weight': 106}. Best is trial 0 with value: 7.793700683273691e+73.


...

[0]	validation_0-rmse:0.11476
[1]	validation_0-rmse:0.09227
[2]	validation_0-rmse:0.08796
[3]	validation_0-rmse:0.08129
[4]	validation_0-rmse:0.08087
[5]	validation_0-rmse:0.07961
[6]	validation_0-rmse:0.08059
[7]	validation_0-rmse:0.07963
[8]	validation_0-rmse:0.08121
[9]	validation_0-rmse:0.07836
[10]	validation_0-rmse:0.07857
[11]	validation_0-rmse:0.07896
[12]	validation_0-rmse:0.07876
[13]	validation_0-rmse:0.07917
[14]	validation_0-rmse:0.07845
[15]	validation_0-rmse:0.07885
[16]	validation_0-rmse:0.07893
[17]	validation_0-rmse:0.07880
[18]	validation_0-rmse:0.07934
[19]	validation_0-rmse:0.07882
[20]	validation_0-rmse:0.07860
[21]	validation_0-rmse:0.07876
[22]	validation_0-rmse:0.07905
[23]	validation_0-rmse:0.07928
[24]	validation_0-rmse:0.07845
[25]	validation_0-rmse:0.07927
[26]	validation_0-rmse:0.07878
[27]	validation_0-rmse:0.07697
[28]	validation_0-rmse:0.07705
[29]	validation_0-rmse:0.07704
[30]	validation_0-rmse:0.07770
[31]	validation_0-rmse:0.07763
[32]	validation_0-

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[18:11:14] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"




[57]	validation_0-rmse:0.07659
[58]	validation_0-rmse:0.07643
[59]	validation_0-rmse:0.07676
[60]	validation_0-rmse:0.07827
[61]	validation_0-rmse:0.07695
[62]	validation_0-rmse:0.07610
[63]	validation_0-rmse:0.07666
[64]	validation_0-rmse:0.07677
[65]	validation_0-rmse:0.07711
[66]	validation_0-rmse:0.07636
[67]	validation_0-rmse:0.07616
[68]	validation_0-rmse:0.07642
[69]	validation_0-rmse:0.07630
[70]	validation_0-rmse:0.07585
[71]	validation_0-rmse:0.07624
[72]	validation_0-rmse:0.07651
[73]	validation_0-rmse:0.07614
[74]	validation_0-rmse:0.07577
[75]	validation_0-rmse:0.07645
[76]	validation_0-rmse:0.07656
[77]	validation_0-rmse:0.07610
[78]	validation_0-rmse:0.07614
[79]	validation_0-rmse:0.07612
[80]	validation_0-rmse:0.07587
[81]	validation_0-rmse:0.07651
[82]	validation_0-rmse:0.07615
[83]	validation_0-rmse:0.07636
[84]	validation_0-rmse:0.07718
[85]	validation_0-rmse:0.07611
[86]	validation_0-rmse:0.07624
[87]	validation_0-rmse:0.07662
[88]	validation_0-rmse:0.07659
[89]	val

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[18:11:15] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"


[I 2024-02-12 18:11:15,680] Trial 14 finished with value: 0.0056983972867274855 and parameters: {'lambda': 0.009151752384274067, 'alpha': 0.02965814872846869, 'colsample_bytree': 0.5, 'subsample': 0.7, 'learning_rate': 1, 'max_depth': 3, 'random_state': 243123, 'min_child_weight': 108}. Best is trial 14 with value: 0.0056983972867274855.


{'lambda': 0.009151752384274067,
 'alpha': 0.02965814872846869,
 'colsample_bytree': 0.5,
 'subsample': 0.7,
 'learning_rate': 1,
 'max_depth': 3,
 'random_state': 243123,
 'min_child_weight': 108}

In [69]:
find_params.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_random_state,params_subsample,state
0,0,7.793701e+73,2024-02-12 18:10:56.955197,2024-02-12 18:10:57.996544,0 days 00:00:01.041347,0.004511,0.7,6.223113,8.00000,10,106,30,0.4,COMPLETE
1,1,2.106883e-02,2024-02-12 18:10:57.999682,2024-02-12 18:10:59.075872,0 days 00:00:01.076190,0.000113,0.2,0.820226,1.00000,7,139,243123,0.4,COMPLETE
2,2,2.105384e-02,2024-02-12 18:10:59.077510,2024-02-12 18:11:00.199116,0 days 00:00:01.121606,0.256390,0.2,0.000101,0.00001,7,163,3454,0.3,COMPLETE
3,3,1.895312e-02,2024-02-12 18:11:00.205710,2024-02-12 18:11:01.822803,0 days 00:00:01.617093,0.000195,0.3,0.001257,0.00030,4,37,3454,0.9,COMPLETE
4,4,2.105673e-02,2024-02-12 18:11:01.825167,2024-02-12 18:11:03.153298,0 days 00:00:01.328131,0.000120,0.2,0.274124,1.00000,9,138,10,0.6,COMPLETE
5,5,5.857839e-03,2024-02-12 18:11:03.156178,2024-02-12 18:11:04.255758,0 days 00:00:01.099580,0.638775,0.5,0.001580,1.00000,3,118,243123,0.9,COMPLETE
6,6,2.098841e-02,2024-02-12 18:11:04.260783,2024-02-12 18:11:05.480086,0 days 00:00:01.219303,0.130919,0.9,0.000166,0.00001,7,52,243123,0.4,COMPLETE
7,7,2.105384e-02,2024-02-12 18:11:05.482358,2024-02-12 18:11:06.558808,0 days 00:00:01.076450,0.636651,0.4,0.015487,0.00030,11,78,3454,0.1,COMPLETE
8,8,2.105384e-02,2024-02-12 18:11:06.563818,2024-02-12 18:11:07.733621,0 days 00:00:01.169803,0.002376,0.3,0.052878,0.00800,7,138,3454,0.2,COMPLETE
9,9,1.125622e-02,2024-02-12 18:11:07.739245,2024-02-12 18:11:09.123755,0 days 00:00:01.384510,9.140397,1.0,0.000516,0.00800,12,50,2000,1.0,COMPLETE


In [68]:
best_params={'lambda': 0.009151752384274067,
 'alpha': 0.02965814872846869,
 'colsample_bytree': 0.5,
 'subsample': 0.7,
 'learning_rate': 1,
 'max_depth': 3,
 'random_state': 243123,
 'min_child_weight': 108}

In [70]:
optuna.visualization.plot_optimization_history(find_params)

In [71]:
!python --version

Python 3.10.12


In [72]:

optuna.visualization.plot_slice(find_params)

In [73]:
optuna.visualization.plot_contour(find_params,params=['alpha','lambda'])

In [74]:
model=xgb.XGBRegressor(**best_params)


In [75]:
model.fit(X_train,y_train)

XGBRegressor(alpha=0.02965814872846869, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.5, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=0.009151752384274067,
             learning_rate=1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
             max_leaves=None, min_child_weight=108, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, ...)

In [76]:
y_pred=model.predict(X_test)

In [77]:
r2score=[0,1]
from sklearn.metrics import r2_score

r2_score(y_test,y_pred)

0.7463031624411209

In [78]:
from sklearn.ensemble import RandomForestRegressor
model2=RandomForestRegressor()
model2.fit(X_train,y_train)
y_pred2=model2.predict(X_test)
r2_score(y_test,y_pred2)


0.8193203339984851

In [32]:
!pip show optuna

Name: optuna
Version: 3.5.0
Summary: A hyperparameter optimization framework
Home-page: 
Author: Takuya Akiba
Author-email: 
License: MIT License
        
        Copyright (c) 2018 Preferred Networks, Inc.
        
        Permission is hereby granted, free of charge, to any person obtaining a copy
        of this software and associated documentation files (the "Software"), to deal
        in the Software without restriction, including without limitation the rights
        to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
        copies of the Software, and to permit persons to whom the Software is
        furnished to do so, subject to the following conditions:
        
        The above copyright notice and this permission notice shall be included in all
        copies or substantial portions of the Software.
        
        THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
        IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIE

In [33]:
!python --version

Python 3.10.12


In [79]:
model=xgb.XGBRegressor(**best_params)

In [80]:
model=xgb.XGBRegressor(**best_params)
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
#r2score=[0,1]
from sklearn.metrics import r2_score

r2_score(y_test,y_pred)

from sklearn.ensemble import RandomForestRegressor
model2=RandomForestRegressor()
model2.fit(X_train,y_train)
y_pred2=model2.predict(X_test)
r2_score(y_test,y_pred2)

0.8191534723159557

In [81]:
y_pred=model.predict(X_test)

In [82]:
#r2score=[0,1]
from sklearn.metrics import r2_score

r2_score(y_test,y_pred)

0.7463031624411209

In [83]:
from sklearn.ensemble import RandomForestRegressor
model2=RandomForestRegressor()
model2.fit(X_train,y_train)
y_pred2=model2.predict(X_test)
r2_score(y_test,y_pred2)

0.8234524776894558

In [84]:
import os
import logging

# Specify the directory and file
dir_path = r'/content/logs/'
log_file = 'system.txt'
full_path = os.path.join(dir_path, log_file)
print(full_path)

/content/logs/system.txt


In [85]:
# Check if the directory exists and create it if necessary
os.makedirs(dir_path, exist_ok=True)

In [86]:
# Set up logging
# Get a logger instance (this will fetch the root logger)
logger = logging.getLogger()

In [87]:
# Set the level of the logger to CRITICAL
# This means it will handle events of level CRITICAL and above
logger.setLevel(logging.CRITICAL)

In [88]:
# Create a FileHandler instance to write logs to a file
handler = logging.FileHandler(full_path)

In [89]:

# Add the handler to the logger
# This connects the logger to the handler so that logs get written to the file
logger.addHandler(handler)


In [90]:
def LetUsCheckSystem(sys):
    if sys != 'OK':
        logging.critical('System failure: %s', sys)

LetUsCheckSystem('You need to handle the issue now')
handler.close()

CRITICAL:root:System failure: You need to handle the issue now


In [91]:
import os
import logging

# Specify the directory and file
dir_path = r'/content/logs/'
log_file = 'system.txt'
full_path = os.path.join(dir_path, log_file)

# Check if the directory exists and create it if necessary
os.makedirs(dir_path, exist_ok=True)

# Set up logging
# Get a logger instance (this will fetch the root logger)
logger = logging.getLogger()

# Set the level of the logger to CRITICAL
# This means it will handle events of level CRITICAL and above
logger.setLevel(logging.CRITICAL)

# Create a FileHandler instance to write logs to a file
handler = logging.FileHandler(full_path)

# Set the format of the logs using a Formatter
# This format includes the log timestamp, log level and log message
handler.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s:%(message)s'))

# Add the handler to the logger
# This connects the logger to the handler so that logs get written to the file
logger.addHandler(handler)




In [92]:
def LetUsCheckSystem(sys):
    if sys != 'OK':
        logging.critical('System failure: %s', sys)

LetUsCheckSystem('You need to handle the issue now')
handler.close()

CRITICAL:root:System failure: You need to handle the issue now
